In [1]:
#jenna forecasting をCT 画像で置き換えてみよう。
#サーキット（細線）の上を辿って、その周りのパッチ（例えば32*32画素）をベクトルとして入力として
#出力も同じパッチを表すベクトルにする。
#入力は０〜２５５の整数値のまま要素にするか、０から１の範囲に正規化する。
#出力の各要素はそれぞれの画素が中心（現時点でのサーキットの位置）と同じクラスに属する場合は１
#属さない場合は０にする。

import keras
keras.__version__

'2.4.3'

In [36]:
import os

csv_path = '/Users/tamurataito/Documents/datasets/jena_climate_2009_2016.csv'


f = open(csv_path)
data = f.read()
f.close()

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]

float_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]]
    float_data[i, :] = values


In [3]:
#現時点でのサーキットの位置を中心とする周りのパッチをsamples
#そのパッチをクラス分けした結果のパッチをtargetsとして配列を返すgeneratorをつくる。
#ただし、パッチの大きさは花弁の分岐点が全ておさまる程度の大きさにする。
#パッチが画像の端に達した場合の処理も考える。
#サーキットのスタート地点を適当に選んでnumber_of_steps_on_circuitだけ進む過程での
#周りのパッチの情報をtargetsとしてかえす。

def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=6):

        yield samples, targets

IndentationError: unexpected indent (<ipython-input-3-eaf657b97260>, line 32)

In [41]:
# jenna forecastingの場合の入力は以下の通り
temp_input = next(train_gen)
temp_input[0].shape

(128, 240, 14)

In [ ]:
#CTセグメンテーションの場合は入力を
#(batch_size, number_of_steps_on_circuit, batch_dim)
#にする。

In [17]:
temp_input[1].shape

(128,)

In [5]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import Adam

patch_dim = 32*32 #パッチをベクトルに引き伸ばした時の次元
intermediate_dim = 1024 #中間層の次元（任意）


model = Sequential()
model.add(layers.LSTM(intermediate_dim,
                     dropout=0.1,
                     recurrent_dropout=0.5,
                     return_sequences=True,
                     input_shape=(None, patch_dim)))

model.add(layers.Dense(patch_dim))

model.compile(optimizer=Adam(), loss='mae')


In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 1024)        8392704   
_________________________________________________________________
dense (Dense)                (None, None, 1024)        1049600   
Total params: 9,442,304
Trainable params: 9,442,304
Non-trainable params: 0
_________________________________________________________________


In [28]:
history = model.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=40,
                              validation_data=val_gen,
                              validation_steps=val_steps)

Epoch 1/40
111/500 [=====>........................] - ETA: 1:53 - loss: 10.3941

KeyboardInterrupt: 